# *Differential Gene Expression Analysis using edgeR* 
Info | Value
---- | ----
Implemented by | Elucidata
Docker | RNASeq Downstream:For transcriptomics analysis
Tag(s) | edgeR - differential expression - pathway analysis

## Initiate and configure the notebook

In [ ]:
# please do not modify
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [ ]:
!sudo pip3 install polly-python --quiet

In [ ]:
restartkernel() #Pause for a few seconds before the kernel is refreshed

In [ ]:
# please do not modify
from IPython.display import HTML
HTML('''<script type="text/javascript"> Jupyter.notebook.kernel.execute("url = '" + window.location + "'", {}, {}); </script>''')

## Fetch OmixAtlas ID and Dataset ID

- **OmixAtlas ID**: Target repository identifier which is required for downloading the dataset
- **Dataset ID**: Identifier for the dataset on the atlas which is to be analysed 

In [ ]:
import urllib.parse as urlparse
from urllib.parse import parse_qs

parsed_url     = urlparse.urlparse(url)
repo_vars_list = [parse_qs(parsed_url.query).get(query_url)[0] for query_url in ['repo_id', 'repo_name', 'dataset_id']]
repo_id        = repo_vars_list[0]
dataset_id     = repo_vars_list[2]

## Download dataset from the OmixAtlas

In [ ]:
from polly.omixatlas import OmixAtlas
import os

In [ ]:
omix_atlas = OmixAtlas(os.environ['POLLY_REFRESH_TOKEN'])

In [ ]:
def download_dataset(repo_id, dataset_id):
    """
    Downloads a single dataset with given repo_id and dataset_id
    """
    file_name = f"{dataset_id}.gct"
    data = omix_atlas.download_data(repo_id, dataset_id)
    url = data.get('data').get('attributes').get('download_url')
    status = os.system(f"wget -O '{file_name}' '{url}'")
    if status == 0:
        print(f"{file_name}: downloaded successfully")
    else:
        raise Exception("Download not successful")

In [ ]:
download_dataset(repo_id, dataset_id)

## Read dataset

In [ ]:
%get dataset_id --from python3
dataset_id

In [ ]:
library(mapGCT)

In [ ]:
gctFile <- paste0(dataset_id, '.gct')
gctObj  <- parse_gct(gctFile)

In [ ]:
counts  <- gctObj@mat
coldata <- gctObj@cdesc
rowdata <- gctObj@rdesc

In [ ]:
dim(counts)
head(counts)

In [ ]:
dim(coldata)
head(coldata)

In [ ]:
dim(rowdata)
head(rowdata)

## Load libraries

In [ ]:
suppressMessages(library(edgeR))
suppressMessages(library(mapGCT))
suppressMessages(library(ggplot2))
suppressMessages(library(ggrepel))
suppressMessages(library(dplyr))
suppressMessages(library(clusterProfiler))
suppressMessages(library(org.Hs.eg.db))

## Define parameters

In [ ]:
%get dataset_id --from python3
dataset_id

In [ ]:
params = list(
    'WORKSPACE' = 8526,
    'ANALYSIS_NAME' = paste0(dataset_id,'_CASE_VS_CONTROL'),
    'COHORT_COL' = 'kw_curated_cell_type',
    'COHORT_CONTROL' = 'HSC',
    'COHORT_CASE' = 'CMP',
    'GSEA_GMT_FILEPATH' = 'polly://data/gmts/h.all.v7.5.1.symbols.gmt',
    'OUTPUT_DIR' = 'polly://results/example',
    'REPORT_FILEPATH' = 'polly://reports/differential_analysis_sample_report.html'
)
params

## Define plot theme

In [ ]:
## Define theme for all plots
custom_theme <- function(base_size=10){
    ggplot2::theme(
        legend.position = "right", legend.direction = "vertical", 
        panel.grid.major = element_blank(),	# major grids included
        panel.grid.minor = element_blank(),	# no minor grids
        plot.title = element_text(color="black", size=base_size*1.5, hjust = 0.5, face="bold"),
        panel.border = element_blank(), panel.background = element_blank(), # no borders and background color
        legend.text = element_text(size = base_size*1.0, face = "bold"),
        legend.title = element_text(size = base_size*1.5, face="bold"),
        axis.title = element_text(colour="black", size = base_size*1.5, face = "bold"), # axis title 
        axis.text.x = element_text(colour="black", size = base_size*1.0, margin=unit(c(0.5,0.5,0.1,0.1), "cm"), face = "bold"), # x-axis text in fontsize 10
        axis.text.y = element_text(colour="black", size = base_size*1.0, margin=unit(c(0.5,0.5,0.1,0.1), "cm"), face = "bold"), # y-axis text in fontsize 10
    ) 
}

## Subset the data for cohorts of interest

In [ ]:
samples_to_keep <- row.names(coldata[coldata[,params$COHORT_COL] %in% c(params$COHORT_CONTROL,params$COHORT_CASE),])
length(samples_to_keep)

In [ ]:
counts_subset <- counts[,samples_to_keep]
coldata_subset<- coldata[samples_to_keep,]
dim(counts_subset)
dim(coldata_subset)

## Create DGEList

In [ ]:
all(colnames(counts_subset) == rownames(coldata_subset))

In [ ]:
y <- DGEList(counts=counts_subset, samples = coldata_subset, group = coldata_subset[, params$COHORT_COL])
head(y$samples)

## Filter out low expression genes

In [ ]:
keep_genes <- filterByExpr(y)
y <- y[keep_genes, ]
dim(y$counts)

## Normalisation

In [ ]:
y <- calcNormFactors(y)
head(y$samples)

In [ ]:
norm.data <- cpm(y, log = TRUE)
head(norm.data)

## PCA

In [ ]:
compute_pca <- function(input_matrix, metadata, ntop_variable_genes = 100){
    input_matrix <- as.data.frame(input_matrix)
    input_matrix$mad <- apply(input_matrix, 1, mad)
    input_matrix <- input_matrix[order(input_matrix$mad, decreasing = T), ]
    input_matrix <- input_matrix[1:ntop_variable_genes, ]
    input_matrix$mad <- NULL
    
    PCAObj <- prcomp(as.data.frame(t(input_matrix)), scale = T)
    PCAObj_Summary <- summary(PCAObj)
    PCA_scores <- data.frame(PCAObj$x, metadata)
    
    return(
        list(
            'scores' = PCA_scores,
            'summary' = PCAObj_Summary
        )
    )
}

In [ ]:
pca_plot <- function(pca, cohortCol, pc_x='PC1', pc_y='PC2', title='PCA', subtitle=''){
    require(ggplot2)
    require(ggsci)
    
    p <- ggplot(pca$scores, aes_string(x = pc_x, y = pc_y, fill = cohortCol)) + 
      geom_point(shape = 21, size = 7, alpha = 0.7) + 
      labs(title = title, subtitle = subtitle,
           x = paste(pc_x, '(', round(pca$summary$importance[2,pc_x]*100, 2), '%)'),
           y = paste(pc_y, '(', round(pca$summary$importance[2,pc_y]*100, 2), '%)'), fill = cohortCol) + 
        scale_fill_brewer(palette="Dark2")+
      custom_theme(base_size = 10)+
      theme(axis.line = element_line(size = 1, colour = "black"))
    p
}

In [ ]:
pca <- compute_pca(norm.data, coldata_subset, nrow(norm.data))
head(pca$scores)

In [ ]:
options(repr.plot.width=12, repr.plot.height=7)
pca_plot(pca, params$COHORT_COL, pc_x='PC1', pc_y='PC2', title='PCA')

## Define the design matrix

In [ ]:
group <- y$samples$group
design <- model.matrix(~ group)
head(design)

## Estimate dispersion

In [ ]:
!sudo Rscript -e "install.packages('statmod', repos='http://cran.us.r-project.org')"

In [ ]:
y <- estimateDisp(y, design, robust=TRUE)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 7)
plotBCV(y)

## Differential expression

In [ ]:
et <- exactTest(y, pair = c(params$COHORT_CONTROL, params$COHORT_CASE))
res <- topTags(et, n = nrow(y$counts))$table
res <- res[order(res$FDR), ]
head(res) 

In [ ]:
print(paste0('No. of genes with logFC > 0 and FDR < 0.05: ', nrow(res[(res$logFC>0)&(res$FDR<0.05),])))
print(paste0('No. of genes with logFC < 0 and FDR < 0.05: ', nrow(res[(res$logFC<0)&(res$FDR<0.05),])))

In [ ]:
mkdir -p results/

In [ ]:
write.csv(res, 'results/der.csv')

## Volcano plot

In [ ]:
volcano_plot <- function(de_result, log2fc_cutoff=1.0, padj_cutoff=0.05, genes_to_label = NULL, title='Volcano', 
                         base_size = 10) {
    
    # Add logical vector as a column (significant) to the res_tableOE
    de_result$significance <- "non-DEG"
    de_result[(abs(de_result$logFC) > log2fc_cutoff) & (de_result$FDR <= padj_cutoff), "significance"] <- "DEG"
    de_result$significance <- factor(de_result$significance, levels=c('DEG','non-DEG'))
    

    ## Create a column to indicate which genes to label
    de_result$genelabels <- ""
    de_result[genes_to_label,'genelabels'] <- genes_to_label

    ## Volcano plot
    p <- ggplot(de_result %>% arrange(desc(significance)), aes(x = logFC, y = -log10(FDR), fill = significance, color = significance, 
                label=genelabels)) +
            geom_point(shape = 21, alpha = 1) +
            geom_hline(yintercept=-log10(padj_cutoff), linetype="dashed", color = "grey") + 
            geom_vline(xintercept=-log2fc_cutoff, linetype="dashed", color = "grey") + 
            geom_vline(xintercept=log2fc_cutoff, linetype="dashed", color = "grey") + 
            geom_text_repel(min.segment.length = 0, color='black', force = 0.05) + 
            ggtitle(title)+
            xlab("log2FC") + 
            ylab("-log10 (padj)") +
            scale_color_manual(values=c("#FF0000", "#dbd9d9"))+
            scale_fill_manual(values=c("#FF0000", "#dbd9d9"))+
            theme(legend.position = "right", legend.direction = "vertical")+
            custom_theme(base_size)+
            theme(axis.line = element_line(size = 1, colour = "black"))
    p
}

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
genes_to_label <- row.names(res[1:5, ])
p_volcano <- volcano_plot(res, log2fc_cutoff=1, padj_cutoff=0.05, title=params$ANALYSIS_NAME, genes_to_label = genes_to_label)
p_volcano

## Gene Ontology Biological Processes on upregulated genes

In [ ]:
ontology = 'BP'
gene_direction = 'up'
fdr_cutoff = 0.05
logFC_cutoff = 1.0

## Create background dataset for hypergeometric testing using all genes tested for significance in the results                 
all_genes <- row.names(res)

## Extract significant results
sig_genes <- row.names(res[(res$FDR < fdr_cutoff), ])
if(gene_direction=='up'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC > logFC_cutoff), ])
}
if(gene_direction=='down'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC < -logFC_cutoff), ])
}

## Run GO enrichment analysis 
ego.bp.up <- enrichGO(gene = sig_genes, 
                   universe = all_genes,
                   keyType = "SYMBOL",
                   OrgDb = org.Hs.eg.db, 
                   ont = ontology, 
                   pAdjustMethod = "BH", 
                   qvalueCutoff = 0.05, 
                   readable = FALSE)

## Store the results in a dataframe
ego.bp.up.cluster_summary <- data.frame(ego.bp.up)
dim(ego.bp.up.cluster_summary)
head(ego.bp.up.cluster_summary, 5)

In [ ]:
## Dotplot
if(nrow(ego.bp.up.cluster_summary) > 0){
    showCategory <- 30
    if (nrow(ego.bp.up.cluster_summary) < 30){
        showCategory <- nrow(ego.bp.up.cluster_summary)
    }
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.bp.up.dotplot <- dotplot(ego.bp.up, showCategory=showCategory)
    ego.bp.up.dotplot +
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    custom_theme()
}

In [ ]:
## Enrichment map
if(nrow(ego.bp.up.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.bp.up.emapplot <- emapplot(ego.bp.up, showCategory = showCategory)
    ego.bp.up.emapplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

In [ ]:
if(nrow(ego.bp.up.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.bp.up.goplot <- goplot(ego.bp.up)
    ego.bp.up.goplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

## Gene Ontology Biological Processes on downregulated genes

In [ ]:
ontology = 'BP'
gene_direction = 'down'
fdr_cutoff = 0.05
logFC_cutoff = 1.0

## Create background dataset for hypergeometric testing using all genes tested for significance in the results                 
all_genes <- row.names(res)

## Extract significant results
sig_genes <- row.names(res[(res$FDR < fdr_cutoff), ])
if(gene_direction=='up'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC > logFC_cutoff), ])
}
if(gene_direction=='down'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC < -logFC_cutoff), ])
}

## Run GO enrichment analysis 
ego.bp.down <- enrichGO(gene = sig_genes, 
                   universe = all_genes,
                   keyType = "SYMBOL",
                   OrgDb = org.Hs.eg.db, 
                   ont = ontology, 
                   pAdjustMethod = "BH", 
                   qvalueCutoff = 0.05, 
                   readable = FALSE)

## Store the results in a dataframe
ego.bp.down.cluster_summary <- data.frame(ego.bp.down)
dim(ego.bp.down.cluster_summary)
head(ego.bp.down.cluster_summary, 5)

In [ ]:
## Dotplot
if(nrow(ego.bp.down.cluster_summary) > 0){
    showCategory <- 30
    if (nrow(ego.bp.down.cluster_summary) < 30){
        showCategory <- nrow(ego.bp.down.cluster_summary)
    }
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.bp.down.dotplot <- dotplot(ego.bp.down, showCategory=showCategory)
    ego.bp.down.dotplot +
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    custom_theme()
}

In [ ]:
## Enrichment map
if(nrow(ego.bp.down.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.bp.down.emapplot <- emapplot(ego.bp.down, showCategory = showCategory)
    ego.bp.down.emapplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

In [ ]:
if(nrow(ego.bp.down.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.bp.down.goplot <- goplot(ego.bp.down)
    ego.bp.down.goplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

## Gene Ontology Molecular Functions on upregulated genes

In [ ]:
ontology = 'MF'
gene_direction = 'up'
fdr_cutoff = 0.05
logFC_cutoff = 1.0

## Create background dataset for hypergeometric testing using all genes tested for significance in the results                 
all_genes <- row.names(res)

## Extract significant results
sig_genes <- row.names(res[(res$FDR < fdr_cutoff), ])
if(gene_direction=='up'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC > logFC_cutoff), ])
}
if(gene_direction=='down'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC < -logFC_cutoff), ])
}

## Run GO enrichment analysis 
ego.mf.up <- enrichGO(gene = sig_genes, 
                   universe = all_genes,
                   keyType = "SYMBOL",
                   OrgDb = org.Hs.eg.db, 
                   ont = ontology, 
                   pAdjustMethod = "BH", 
                   qvalueCutoff = 0.05, 
                   readable = FALSE)

## Store the results in a dataframe
ego.mf.up.cluster_summary <- data.frame(ego.mf.up)
dim(ego.mf.up.cluster_summary)
head(ego.mf.up.cluster_summary, 5)

In [ ]:
## Dotplot
if(nrow(ego.mf.up.cluster_summary) > 0){
    showCategory <- 30
    if (nrow(ego.mf.up.cluster_summary) < 30){
        showCategory <- nrow(ego.cluster_summary)
    }
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.mf.up.dotplot <- dotplot(ego.mf.up, showCategory=showCategory)
    ego.mf.up.dotplot +
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    custom_theme()
}

In [ ]:
## Enrichment map
if(nrow(ego.mf.up.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.mf.up.emapplot <- emapplot(ego.mf.up, showCategory = showCategory)
    ego.mf.up.emapplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

In [ ]:
if(nrow(ego.mf.up.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.mf.up.goplot <- goplot(ego.mf.up)
    ego.mf.up.goplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

## Gene Ontology Molecular Functions on downregulated genes

In [ ]:
ontology = 'MF'
gene_direction = 'down'
fdr_cutoff = 0.05
logFC_cutoff = 1.0

## Create background dataset for hypergeometric testing using all genes tested for significance in the results                 
all_genes <- row.names(res)

## Extract significant results
sig_genes <- row.names(res[(res$FDR < fdr_cutoff), ])
if(gene_direction=='up'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC > logFC_cutoff), ])
}
if(gene_direction=='down'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC < -logFC_cutoff), ])
}

## Run GO enrichment analysis 
ego.mf.down <- enrichGO(gene = sig_genes, 
                   universe = all_genes,
                   keyType = "SYMBOL",
                   OrgDb = org.Hs.eg.db, 
                   ont = ontology, 
                   pAdjustMethod = "BH", 
                   qvalueCutoff = 0.05, 
                   readable = FALSE)

## Store the results in a dataframe
ego.mf.down.cluster_summary <- data.frame(ego.mf.down)
dim(ego.mf.down.cluster_summary)
head(ego.mf.down.cluster_summary, 5)

In [ ]:
## Dotplot
if(nrow(ego.mf.down.cluster_summary) > 0){
    showCategory <- 30
    if (nrow(ego.mf.down.cluster_summary) < 30){
        showCategory <- nrow(ego.mf.down.cluster_summary)
    }
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.mf.down.dotplot <- dotplot(ego.mf.down, showCategory=showCategory)
    ego.mf.down.dotplot +
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    custom_theme()
}

In [ ]:
## Enrichment map
if(nrow(ego.mf.down.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.mf.down.emapplot <- emapplot(ego.mf.down, showCategory = showCategory)
    ego.mf.down.emapplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

In [ ]:
if(nrow(ego.mf.down.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.mf.down.goplot <- goplot(ego.mf.down)
    ego.mf.down.goplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

## Gene Ontology Cellular Components on upregulated genes

In [ ]:
ontology = 'CC'
gene_direction = 'up'
fdr_cutoff = 0.05
logFC_cutoff = 1.0

## Create background dataset for hypergeometric testing using all genes tested for significance in the results                 
all_genes <- row.names(res)

## Extract significant results
sig_genes <- row.names(res[(res$FDR < fdr_cutoff), ])
if(gene_direction=='up'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC > logFC_cutoff), ])
}
if(gene_direction=='down'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC < -logFC_cutoff), ])
}

## Run GO enrichment analysis 
ego.cc.up <- enrichGO(gene = sig_genes, 
                   universe = all_genes,
                   keyType = "SYMBOL",
                   OrgDb = org.Hs.eg.db, 
                   ont = ontology, 
                   pAdjustMethod = "BH", 
                   qvalueCutoff = 0.05, 
                   readable = FALSE)

## Store the results in a dataframe
ego.cc.up.cluster_summary <- data.frame(ego.cc.up)
dim(ego.cc.up.cluster_summary)
head(ego.cc.up.cluster_summary, 5)

In [ ]:
## Dotplot
if(nrow(ego.cc.up.cluster_summary) > 0){
    showCategory <- 30
    if (nrow(ego.cc.up.cluster_summary) < 30){
        showCategory <- nrow(ego.cc.up.cluster_summary)
    }
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.cc.up.dotplot <- dotplot(ego.cc.up, showCategory=showCategory)
    ego.cc.up.dotplot +
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    custom_theme()
}

In [ ]:
## Enrichment map
if(nrow(ego.cc.up.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.cc.up.emapplot <- emapplot(ego.cc.up, showCategory = showCategory)
    ego.cc.up.emapplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

In [ ]:
if(nrow(ego.cc.up.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.cc.up.goplot <- goplot(ego.cc.up)
    ego.cc.up.goplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

## Gene Ontology Cellular Components on downregulated genes

In [ ]:
ontology = 'CC'
gene_direction = 'down'
fdr_cutoff = 0.05
logFC_cutoff = 1.0

## Create background dataset for hypergeometric testing using all genes tested for significance in the results                 
all_genes <- row.names(res)

## Extract significant results
sig_genes <- row.names(res[(res$FDR < fdr_cutoff), ])
if(gene_direction=='up'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC > logFC_cutoff), ])
}
if(gene_direction=='down'){
    sig_genes <- row.names(res[(res$FDR < fdr_cutoff) & (res$logFC < -logFC_cutoff), ])
}

## Run GO enrichment analysis 
ego.cc.down <- enrichGO(gene = sig_genes, 
                   universe = all_genes,
                   keyType = "SYMBOL",
                   OrgDb = org.Hs.eg.db, 
                   ont = ontology, 
                   pAdjustMethod = "BH", 
                   qvalueCutoff = 0.05, 
                   readable = FALSE)

## Store the results in a dataframe
ego.cc.down.cluster_summary <- data.frame(ego.cc.down)
dim(ego.cc.down.cluster_summary)
head(ego.cc.down.cluster_summary, 5)

In [ ]:
## Dotplot
if(nrow(ego.cc.down.cluster_summary) > 0){
    showCategory <- 30
    if (nrow(ego.cc.down.cluster_summary) < 30){
        showCategory <- nrow(ego.cc.down.cluster_summary)
    }
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.cc.down.dotplot <- dotplot(ego.cc.down, showCategory=showCategory)
    ego.cc.down.dotplot +
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    custom_theme()
}

In [ ]:
## Enrichment map
if(nrow(ego.cc.down.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.cc.down.emapplot <- emapplot(ego.cc.down, showCategory = showCategory)
    ego.cc.down.emapplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

In [ ]:
if(nrow(ego.cc.down.cluster_summary) > 0){
    options(repr.plot.width = 16, repr.plot.height = 8)

    ego.cc.down.goplot <- goplot(ego.cc.down)
    ego.cc.down.goplot+
    ggtitle(paste0(params$ANALYSIS_NAME, ' GO', ontology, ' ',gene_direction, 'regulated genes'))+
    ggplot2::theme(
        plot.title = element_text(color="black", size=15, hjust = 0.5, face="bold")
    )
}

## GSEA

### Download Hallmark gene sets

In [ ]:
cmd <- paste0("polly files copy --workspace-id ",params$WORKSPACE, " -s ",params$GSEA_GMT_FILEPATH, " -d gsea_pathways.gmt")
system(cmd, intern = TRUE)

In [ ]:
library(fgsea)
ranks <- res$logFC
names(ranks) <- row.names(res)
gsea_pathways <- gmtPathways('gsea_pathways.gmt')

fgseaRes <- fgsea(pathways = gsea_pathways, 
                  stats    = ranks,
                  minSize  = 15,
                  maxSize  = 500,
                  nperm = 100)
fgseaRes <- fgseaRes[order(fgseaRes$ES), ]

dim(fgseaRes)
head(fgseaRes)

In [ ]:
top_gsea_results <- as.data.frame(rbind(head(fgseaRes, 10), tail(fgseaRes, 10)))
top_gsea_results <- unique(top_gsea_results)
top_gsea_results[,1:7]

In [ ]:
min_ngenes <- 2^ceiling(log2(min(top_gsea_results$size)))
max_ngenes <- 2^ceiling(log2(max(top_gsea_results$size)))

p <- ggplot(top_gsea_results, aes(x = ES, y = pathway, fill=padj, size=size))+
        geom_point(shape = 21, alpha = 1.0) +
        scale_y_discrete(limits = top_gsea_results$pathway)+
        scale_size_continuous(range = c(.1, 24), name="No. of genes", breaks = seq(min_ngenes, max_ngenes, length.out=5)) + 
        scale_fill_continuous(low="red", high="grey", limits=c(0,0.05)) +
        ggtitle(paste0(params$ANALYSIS_NAME, ' GSEA'))+
        xlab("enrichmentScore") + 
        ylab("Pathway") +
        theme(legend.position = "right", legend.direction = "vertical", # legend positioned at the bottom, horizantal direction,
              axis.line = element_line(size=1, colour = "black"),	# axis line of size 1 inch in black color
              panel.grid.major = element_blank(),	# major grids included
              panel.grid.minor = element_blank(),	# no minor grids
              plot.title = element_text(color="black", size=20, hjust = 0.5, face="bold"),
              panel.border = element_blank(), panel.background = element_blank(), # no borders and background color
              axis.title = element_text(colour="black", size = 20, face = "bold"), # axis title 
              axis.text.x = element_text(colour="black", size = 15, margin=unit(c(0.5,0.5,0.1,0.1), "cm"), face = "bold"), # x-axis text in fontsize 10
              axis.text.y = element_text(colour="black", size = 15, margin=unit(c(0.5,0.5,0.1,0.1), "cm"), face = "bold"), # y-axis text in fontsize 10
              legend.text = element_text(size = 15, face = "bold"),
              legend.title = element_text(size = 15, face = "bold"),
              axis.ticks.length = unit(-0.25, "cm")) # ticks facing inward with 0.25cm length
p

## X2K: Using API

In [ ]:
# Import modules
import requests
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

##### Function to run X2K
### Input: a Python list of gene symbols
### Output: a dictionary containing the results of X2K, ChEA, G2N, KEA.

def run_X2K(input_genes, options={}):
    # Set default options
    all_options = {'included_organisms': 'both',
                       'TF-target gene background database used for enrichment': 'ChEA & ENCODE Consensus',
                       'sort transcription factors by': 'p-value',
                       'min_network_size': 10,
                       'number of top TFs': 10,
                       'path_length': 2,
                       'min_number_of_articles_supporting_interaction': 0,
                       'max_number_of_interactions_per_protein': 200,
                       'max_number_of_interactions_per_article': 100,
                       'enable_BioGRID': True,
                       'enable_IntAct': True,
                       'enable_MINT': True,
                       'enable_ppid': True,
                       'enable_Stelzl': True,
                       'kinase interactions to include': 'kea 2018',
                       'sort kinases by': 'p-value'}

    # Override defaults with options
    all_options.update(options)
    all_options['text-genes'] = '\n'.join(input_genes)

    # Perform request & get response
    res = requests.post(
        'https://maayanlab.cloud/X2K/api',
        files=[(k, (None, v)) for k, v in all_options.items()],
    )

    # Read response
    data = res.json()

    # Convert to dictionary
    x2k_results = {key: json.loads(value) if key != 'input' else value for key, value in data.items()}

    # Clean results
    x2k_results['ChEA'] = x2k_results['ChEA']['tfs']
    x2k_results['G2N'] = x2k_results['G2N']['network']
    x2k_results['KEA'] = x2k_results['KEA']['kinases']
    x2k_results['X2K'] = x2k_results['X2K']['network']

    # Return results
    return x2k_results

### 1. Read DE results

In [ ]:
de = pd.read_csv('results/der.csv', index_col=0)
de.head()

### 2. Get The List Of DEGs For Input To X2K

In [ ]:
x2k_input_genes = de.loc[de['FDR'] < 0.05].index.values
len(x2k_input_genes)

### 3. Run X2K

In [ ]:
x2k_results = run_X2K(x2k_input_genes)

In [ ]:
x2k_results.keys()

### 4. ChEA

In [ ]:
chea = pd.DataFrame(x2k_results['ChEA'])
chea['neg_log10_pval'] = -np.log10(chea['pvalue'])
chea = chea.sort_values(by=['pvalue'])
chea.head()

In [ ]:
chea.to_csv('results/chea.csv')

### 5. KEA

In [ ]:
kea = pd.DataFrame(x2k_results['KEA'])
kea['neg_log10_pval'] = -np.log10(kea['pvalue'])
kea = kea.sort_values(by=['pvalue'])
kea.head()

In [ ]:
kea.to_csv('results/kea.csv')

### 6. ChEA and KEA visualization

In [ ]:
%get params --from R

In [ ]:
# Initialize the matplotlib figure

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8))
fig.suptitle(params['ANALYSIS_NAME']+' X2K')
sns.set_color_codes("pastel")
sns.set_theme(style="white")
sns.despine(left=True)

sns.barplot(y="name", x="neg_log10_pval", data=chea.iloc[0:10], color="brown", ax=ax1)
ax1.set(ylabel="Transcription factors", xlabel="-log10(pval)")

sns.barplot(y="name", x="neg_log10_pval", data=kea.iloc[0:10], color="b", ax=ax2)
ax2.set(ylabel="Kinases", xlabel="-log10(pval)")

## Write results to workspace

In [ ]:
write.csv(ego.bp.up.cluster_summary, 'results/ego_bp_up_cluster_summary.csv')
write.csv(ego.bp.down.cluster_summary, 'results/ego_bp_down_cluster_summary.csv')
write.csv(ego.mf.up.cluster_summary, 'results/ego_mf_up_cluster_summary.csv')
write.csv(ego.mf.down.cluster_summary, 'results/ego_mf_down_cluster_summary.csv')
write.csv(ego.cc.up.cluster_summary, 'results/ego_cc_up_cluster_summary.csv')
write.csv(ego.cc.down.cluster_summary, 'results/ego_cc_down_cluster_summary.csv')

In [ ]:
cmd <- paste0("polly files sync --workspace-id ", params$WORKSPACE,
              " -s results/ -d ", params$OUTPUT_DIR)
system(cmd, intern = TRUE)

## Publish an HTML report

In [ ]:
def get_notebook_name():
    files = os.listdir()
    notebook = [file for file in files if '.ipynb' in file][0]
    return notebook

In [ ]:
notebook_name = get_notebook_name()
notebook_name

In [ ]:
%get notebook_name --from python3
notebook_name

In [ ]:
cmd <- paste0("jupyter nbconvert --to HTML --TemplateExporter.exclude_input=True --no-prompt --output report.html ",
              notebook_name)
system(cmd, intern = TRUE)

In [ ]:
cmd <- paste0("polly files copy --workspace-id ", params$WORKSPACE, 
             " -s report.html",
             " -d ", params$REPORT_FILEPATH)
system(cmd, intern=TRUE)